In [1]:
import os
import itertools
import shutil
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation,Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import load_model

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import numpy as np
import pandas as pd 
from glob import glob
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
images = []
image_path = r'C:\Users\Harrisagar\Desktop\DME\Dataset' # Original file path here
destination_path = r'C:\Users\Harrisagar\Desktop\DME\Data_out' # Processed file path here



# Function for cropping of images
def crop(image_list, indir, outdir):
    for j in image_list:
        img = cv2.imread(indir + '\\'+ j)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        gray = np.asarray(gray)
        r, c  = gray.shape
        thresh = 5
        r1 = 0; c1 = 0; r2 = r; c2 = c

        for i in range(int(r/2), 0, -1):
            if np.mean(gray[i,:]) <= thresh:
                r1 = i
                break

        for i in range(int(c/2), 0, -1):
            if np.mean(gray[:,i]) <= thresh:
                c1 = i
                break

        for i in range(int(r/2), r, 1):
            if np.mean(gray[i,:]) <= thresh:
                r2 = i
                break

        for i in range(int(c/2), c, 1):
            if np.mean(gray[:,i]) <= thresh:
                c2 = i
                break

        crop = img[r1:r2,c1:c2]
        cv2.imwrite(os.path.join(outdir , j), crop)

crop(allimages, r'')

In [ ]:
def resize(piclist, indir, outdir):
    for i in piclist:
        image = cv2.imread(indir + '\\'+ i)
        image_res = imutils.resize(image, (512, 512), interpolation = cv2.INTER_NEAREST)
        cv2.imwrite(os.path.join(outdir, i), image_res)
        
resize(allimages, r'')


In [ ]:
num = len(images)
for i in range(num):
    # label = images[i].split('_')[1]
    temp = os.path.basename(images[i])
    temp1 = os.path.splitext(temp)[0]
    img = cv2.imread(images[i])
    edges = cv2.Canny(img, 100, 210)
    kernel = np.ones((95, 95), np.uint8)
    erosion = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    contours, hierarchy = cv2.findContours(erosion, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnt = contours[0]
    x, y, w, h = cv2.boundingRect(cnt)
    # img = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 3)
    crop_img = img[y:y + h, x:x + w]
    # plt.subplot(1, 3, 1), plt.imshow(img, 'gray')
    # plt.subplot(1, 3, 2), plt.imshow(erosion, 'gray')
    # plt.subplot(1, 3, 3), plt.imshow(crop_img, 'gray')

    cv2.imwrite(destination_path+temp1+'.jpg', crop_img[:, :, 1])

In [ ]:
# This function prints and plots the confusion matrix.
# Normalization can be applied by setting `normalize=True`.
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
                          plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
